In [1]:
!python -V

Python 3.10.9


In [18]:
# import packages
import pandas as pd
import numpy as np

In [3]:
# install pyarrow for reading parquet files
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 7.5 MB/s eta 0:00:0000:0100:01


In [4]:
# import the graphing/visualization packages
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
# import the ml scikit learn modules

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [10]:
# Read the yellow taxi trips data for January 2022

df = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [11]:
# Get number of columns in df
len(df.columns)

19

In [12]:
# Compute a new feature named 'duration' which contains the duration of a ride in minutes
df['duration'] = pd.to_datetime(df.tpep_dropoff_datetime)- pd.to_datetime(df.tpep_pickup_datetime)
df['duration'] = df.duration.apply(lambda td: td.total_seconds()/60)

In [19]:
# Compute the standard deviation of trip duration for January

df['duration'].std()

46.44530513776802

In [20]:
# Removing outliers from duration
records_before = len(df.duration)
records_after = len(df[(df.duration>=1) & (df.duration<=60)])
print("Before removing outliers:", records_before)
print("After removing outliers:", records_after)
print("% after:", 100*records_after/records_before)

Before removing outliers: 2463931
After removing outliers: 2421440
% after: 98.27547930522405


In [21]:
# Removing duration outliers from the dataset
df = df[ (df.duration>=1) & (df.duration<=60) ]
len(df)

2421440

In [22]:
# Apply one-hot encoding to pickup and dropoff location IDs

categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

/var/folders/q6/4vk5yrts387cp_3qs430rhqc0000gn/T/ipykernel_8540/736078973.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [23]:
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [24]:
X_train.shape

(2421440, 515)

In [25]:
# Set the target variable
target = 'duration'
y_train = df[target].values

In [26]:
# Train a plain linear regression model ith default parameters
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [27]:
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

6.986190742248472

In [28]:
def read_dataframe(filename):
    
    if filename.endswith(".csv"):
        df = pd.read_csv(filename)
    elif filename.endswith(".parquet"):
        df = pd.read_parquet(filename)
        
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime  = pd.to_datetime(df.tpep_pickup_datetime)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)
    
    # filter dataframe based on duration
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    # select columns for features
    categorical = ['PULocationID', 'DOLocationID']
    # convert the columns to str type, prepping for one-hot encoding
    df[categorical]= df[categorical].astype(str)
    
    return df

In [29]:
# read in the validation data - February 2022 dataset
df_val = read_dataframe('./data/yellow_tripdata_2022-02.parquet')
df_val.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.0,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25,12.433333
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.0,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25,17.550000
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.5,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25,23.650000
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.0,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00,20.083333
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.5,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25,26.316667


In [30]:
# Get the validation feature data
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df_val[target].values

In [31]:
# Predict on and evaluate the validation data
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.78640662117552